# DiscoGAN vs CycleGAN

DiscoGAN and CycleGAN are both Generative Adversarial Networks (GANs) designed for unsupervised image-to-image translation, but they have some differences in their design and applications. Here’s a detailed comparison of the two.

## Commonalities

- **Unsupervised Learning**: Both models are used for unsupervised learning, where paired training data is not available.
- **Cycle Consistency Loss**: Both models use cycle consistency loss to ensure that translating an image to the target domain and back results in the original image.
- **Architecture**: Both employ a pair of generators and a pair of discriminators for their operations.

## Differences

| Aspect | DiscoGAN | CycleGAN |
|--------|----------|----------|
| **Objective** | Focuses on discovering cross-domain relationships and transferring styles between two different domains. | Primarily designed for image-to-image translation tasks where paired examples are not available. |
| **Applications** | Commonly used for style transfer, such as transforming objects or styles between different domains (e.g., faces of different genders, artistic styles). | Broadly used for a variety of tasks including artistic style transfer, photo enhancement, object transfiguration, and more. |
| **Cycle Consistency Implementation** | Uses cycle consistency loss to ensure that the transformation to the target domain and back results in the original image. | Similar use of cycle consistency loss but may differ in implementation specifics, such as the exact loss functions and architectures used. |
| **Loss Functions** | Uses a combination of adversarial loss and cycle consistency loss. The specifics can vary, but typically includes mean squared error (MSE) for cycle consistency. | Uses a combination of adversarial loss and cycle consistency loss, often employing L1 norm for the cycle consistency loss. |
| **Architecture Specifics** | May vary more in practice, with some implementations focusing on specific types of data or transformations. | Typically follows the architecture as proposed in the original CycleGAN paper, which includes the use of instance normalization and residual blocks. |
| **Training Stability** | Often requires careful tuning and may be less stable due to its focus on style transfer, which can introduce more variability. | Generally stable and well-documented, with extensive resources and implementations available. |

## Example Code Snippets

### DiscoGAN Generator

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Activation, Add, BatchNormalization, LeakyReLU, Flatten, Dense
from tensorflow.keras.models import Model

def residual_block(x, filters):
    res = Conv2D(filters, (3, 3), padding='same')(x)
    res = BatchNormalization()(res)
    res = Activation('relu')(res)
    res = Conv2D(filters, (3, 3), padding='same')(res)
    res = BatchNormalization()(res)
    return Add()([res, x])

def build_discogan_generator():
    inputs = Input(shape=(64, 64, 3))
    x = Conv2D(64, (7, 7), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    for _ in range(6):
        x = residual_block(x, 256)
    x = Conv2DTranspose(128, (3, 3), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(64, (3, 3), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(3, (7, 7), padding='same')(x)
    x = Activation('tanh')(x)
    return Model(inputs, x)

def build_discogan_discriminator():
    inputs = Input(shape=(64, 64, 3))
    x = Conv2D(64, (4, 4), strides=2, padding='same')(inputs)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(128, (4, 4), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(256, (4, 4), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(512, (4, 4), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Flatten()(x)
    x = Dense(1, activation='sigmoid')(x)
    return Model(inputs, x)

def build_discogan(generator_A_to_B, generator_B_to_A, discriminator_A, discriminator_B):
    discriminator_A.trainable = False
    discriminator_B.trainable = False
    real_A = Input(shape=(64, 64, 3))
    real_B = Input(shape=(64, 64, 3))
    fake_B = generator_A_to_B(real_A)
    fake_A = generator_B_to_A(real_B)
    recon_A = generator_B_to_A(fake_B)
    recon_B = generator_A_to_B(fake_A)
    valid_A = discriminator_A(fake_A)
    valid_B = discriminator_B(fake_B)
    combined = Model(inputs=[real_A, real_B], outputs=[valid_A, valid_B, recon_A, recon_B])
    combined.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss=['mse', 'mse', 'mae', 'mae'], loss_weights=[1, 1, 10, 10])
    return combined

# Instantiate the generators and discriminators
generator_A_to_B = build_discogan_generator()
generator_B_to_A = build_discogan_generator()
discriminator_A = build_discogan_discriminator()
discriminator_B = build_discogan_discriminator()

# Compile the discriminators
discriminator_A.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='mse', metrics=['accuracy'])
discriminator_B.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='mse', metrics=['accuracy'])

# Instantiate and summarize the DiscoGAN
discogan = build_discogan(generator_A_to_B, generator_B_to_A, discriminator_A, discriminator_B)
discogan.summary()

# Sample data
data_A = np.random.rand(100, 64, 64, 3).astype(np.float32)
data_B = np.random.rand(100, 64, 64, 3).astype(np.float32)

# Training parameters
epochs = 10000
batch_size = 1
patch_size = discriminator_A.output_shape[1]

# Training loop
for epoch in range(epochs):
    for _ in range(data_A.shape[0] // batch_size):
        # Train discriminators with real samples
        idx = np.random.randint(0, data_A.shape[0], batch_size)
        X_real_A, y_real_A = data_A[idx], np.ones((batch_size, patch_size, patch_size, 1))
        X_real_B, y_real_B = data_B[idx], np.ones((batch_size, patch_size, patch_size, 1))
        dA_loss_real = discriminator_A.train_on_batch(X_real_A, y_real_A)
        dB_loss_real = discriminator_B.train_on_batch(X_real_B, y_real_B)

        # Train discriminators with fake samples
        X_fake_A, y_fake_A = generator_B_to_A.predict(X_real_B), np.zeros((batch_size, patch_size, patch_size, 1))
        X_fake_B, y_fake_B = generator_A_to_B.predict(X_real_A), np.zeros((batch_size, patch_size, patch_size, 1))
        dA_loss_fake = discriminator_A.train_on_batch(X_fake_A, y_fake_A)
        dB_loss_fake = discriminator_B.train_on_batch(X_fake_B, y_fake_B)

        # Train generators
        g_loss = discogan.train_on_batch([X_real_A, X_real_B], [y_real_A, y_real_B, X_real_A, X_real_B])

    # Summarize the loss for this epoch
    if (epoch + 1) % 1000 == 0:
        print(f'Epoch {epoch+1}/{epochs}, dA_real_loss={dA_loss_real[0]}, dA_fake_loss={dA_loss_fake[0]}, dB_real_loss={dB_loss_real[0]}, dB_fake_loss={dB_loss_fake[0]}, g_loss={g_loss[0]}')


### CycleGAN Generator

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Activation, Add
from tensorflow.keras.models import Model

class InstanceNormalization(tf.keras.layers.Layer):
    def __init__(self, epsilon=1e-5):
        super(InstanceNormalization, self).__init__()
        self.epsilon = epsilon

    def build(self, input_shape):
        self.gamma = self.add_weight(
            shape=(input_shape[-1],),
            initializer="ones",
            trainable=True,
        )
        self.beta = self.add_weight(
            shape=(input_shape[-1],),
            initializer="zeros",
            trainable=True,
        )

    def call(self, inputs):
        mean, variance = tf.nn.moments(inputs, axes=[1, 2], keepdims=True)
        normalized = (inputs - mean) / tf.sqrt(variance + self.epsilon)
        return self.gamma * normalized + self.beta

# Define the residual block
def residual_block(x, filters):
    res = Conv2D(filters, (3, 3), padding='same')(x)
    res = InstanceNormalization()(res)
    res = Activation('relu')(res)
    res = Conv2D(filters, (3, 3), padding='same')(res)
    res = InstanceNormalization()(res)
    return Add()([res, x])

# Function to build the generator
def build_generator():
    inputs = Input(shape=(64, 64, 3))
    x = Conv2D(64, (7, 7), padding='same')(inputs)
    x = InstanceNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3), strides=2, padding='same')(x)
    x = InstanceNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), strides=2, padding='same')(x)
    x = InstanceNormalization()(x)
    x = Activation('relu')(x)
    for _ in range(6):
        x = residual_block(x, 256)
    x = Conv2DTranspose(128, (3, 3), strides=2, padding='same')(x)
    x = InstanceNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(64, (3, 3), strides=2, padding='same')(x)
    x = InstanceNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(3, (7, 7), padding='same')(x)
    x = Activation('tanh')(x)
    return Model(inputs, x)

# Function to build the discriminator
def build_discriminator():
    inputs = Input(shape=(64, 64, 3))
    x = Conv2D(64, (4, 4), strides=2, padding='same')(inputs)
    x = Activation('relu')(x)
    x = Conv2D(128, (4, 4), strides=2, padding='same')(x)
    x = InstanceNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (4, 4), strides=2, padding='same')(x)
    x = InstanceNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (4, 4), strides=2, padding='same')(x)
    x = InstanceNormalization()(x)
    x = Activation('relu')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    return Model(inputs, x)

# Function to build the combined DiscoGAN model
def build_discogan(generator_A_to_B, generator_B_to_A, discriminator_A, discriminator_B):
    discriminator_A.trainable = False
    discriminator_B.trainable = False
    real_A = Input(shape=(64, 64, 3))
    real_B = Input(shape=(64, 64, 3))
    fake_B = generator_A_to_B(real_A)
    fake_A = generator_B_to_A(real_B)
    recon_A = generator_B_to_A(fake_B)
    recon_B = generator_A_to_B(fake_A)
    valid_A = discriminator_A(fake_A)
    valid_B = discriminator_B(fake_B)
    combined = Model(inputs=[real_A, real_B], outputs=[valid_A, valid_B, recon_A, recon_B])
    combined.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss=['mse', 'mse', 'mae', 'mae'], loss_weights=[1, 1, 10, 10])
    return combined

# Instantiate the generators and discriminators
generator_A_to_B = build_generator()
generator_B_to_A = build_generator()
discriminator_A = build_discriminator()
discriminator_B = build_discriminator()

# Compile the discriminators
discriminator_A.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='mse', metrics=['accuracy'])
discriminator_B.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='mse', metrics=['accuracy'])

# Instantiate and summarize the DiscoGAN
discogan = build_discogan(generator_A_to_B, generator_B_to_A, discriminator_A, discriminator_B)
discogan.summary()

# Sample data
data_A = np.random.rand(100, 64, 64, 3).astype(np.float32)
data_B = np.random.rand(100, 64, 64, 3).astype(np.float32)

# Training parameters
epochs = 10000
batch_size = 1
patch_size = discriminator_A.output_shape[1]

# Training loop
for epoch in range(epochs):
    for _ in range(data_A.shape[0] // batch_size):
        # Train discriminators with real samples
        idx = np.random.randint(0, data_A.shape[0], batch_size)
        X_real_A, y_real_A = data_A[idx], np.ones((batch_size, patch_size, patch_size, 1))
        X_real_B, y_real_B = data_B[idx], np.ones((batch_size, patch_size, patch_size, 1))
        dA_loss_real = discriminator_A.train_on_batch(X_real_A, y_real_A)
        dB_loss_real = discriminator_B.train_on_batch(X_real_B, y_real_B)

        # Train discriminators with fake samples
        X_fake_A, y_fake_A = generator_B_to_A.predict(X_real_B), np.zeros((batch_size, patch_size, patch_size, 1))
        X_fake_B, y_fake_B = generator_A_to_B.predict(X_real_A), np.zeros((batch_size, patch_size, patch_size, 1))
        dA_loss_fake = discriminator_A.train_on_batch(X_fake_A, y_fake_A)
        dB_loss_fake = discriminator_B.train_on_batch(X_fake_B, y_fake_B)

        # Train generators
        g_loss = discogan.train_on_batch([X_real_A, X_real_B], [y_real_A, y_real_B, X_real_A, X_real_B])

    # Summarize the loss for this epoch
    if (epoch + 1) % 1000 == 0:
        print(f'Epoch {epoch+1}/{epochs}, dA_real_loss={dA_loss_real[0]}, dA_fake_loss={dA_loss_fake[0]}, dB_real_loss={dB_loss_real[0]}, dB_fake_loss={dB_loss_fake[0]}, g_loss={g_loss[0]}')


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5             │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_4             │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_1 (Functional) │ (None, 64, 64, 3)      │      7,845,123 │ input_layer_5[0][0],   │
│                           │                        │                │ functional[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional (Functional)   │ (None, 64, 64, 3)      │      7,845,123 │ input_layer_4[0][0],   │
│                           │                        │                │ functional_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_2 (Functional) │ (None, 1)              │      2,766,529 │ functional_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_3 (Functional) │ (None, 1)              │      2,766,529 │ functional[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 21,223,304 (80.96 MB)

 Trainable params: 15,690,246 (59.85 MB)

 Non-trainable params: 5,533,058 (21.11 MB)

/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


## Conclusion

Both DiscoGAN and CycleGAN are powerful tools for unsupervised image-to-image translation, with each having its own strengths and areas of application. Understanding their differences can help you choose the right model for your specific task.